# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [5]:
#login.items()
#login['API Key']

In [7]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [31]:
# set our API call parameters and filename before the first call
location = 'Seattle, WA 98012'
term = 'barbecue'

In [32]:
location.split(',')[0]

'Seattle'

In [33]:
## Specify folder for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [34]:
JSON_FILE

'Data/Seattle-barbecue.json'

### Check if Json File exists and Create it if it doesn't

In [35]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        #create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[ii] {JSON_FILE} already exists.")

[i] Data/Seattle-barbecue.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [36]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [37]:
type(results)

dict

In [38]:
len(results)

3

In [39]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [40]:
results['total']

733

In [41]:
results['region']

{'center': {'longitude': -122.19955444335938, 'latitude': 47.84537662548611}}

In [42]:
results['businesses']

[{'id': 'fU2S1YNmakvTVFC8pIlY9A',
  'alias': 'brileys-bbq-and-grill-lake-forest-park',
  'name': 'Brileys BBQ & Grill',
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/yKeCFDp2U3V_I2UMCxVGkg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/brileys-bbq-and-grill-lake-forest-park?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
  'review_count': 862,
  'categories': [{'alias': 'bbq', 'title': 'Barbeque'},
   {'alias': 'tradamerican', 'title': 'American'}],
  'rating': 4.5,
  'coordinates': {'latitude': 47.73853, 'longitude': -122.29},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '15030 Bothell Way NE',
   'address2': '',
   'address3': '',
   'city': 'Lake Forest Park',
   'zip_code': '98155',
   'country': 'US',
   'state': 'WA',
   'display_address': ['15030 Bothell Way NE', 'Lake Forest Park, WA 98155']},
  'phone': '+12064661589',
  'display_phon

In [43]:
## How many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fU2S1YNmakvTVFC8pIlY9A,brileys-bbq-and-grill-lake-forest-park,Brileys BBQ & Grill,https://s3-media3.fl.yelpcdn.com/bphoto/yKeCFD...,False,https://www.yelp.com/biz/brileys-bbq-and-grill...,862,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.5,"{'latitude': 47.73853, 'longitude': -122.29}",[delivery],$$,"{'address1': '15030 Bothell Way NE', 'address2...",+12064661589,(206) 466-1589,13665.506367
1,ZIRJB-lk5cD4L8GUOca3zw,fat-pig-bbq-edmonds,Fat Pig BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/Xdbg4J...,False,https://www.yelp.com/biz/fat-pig-bbq-edmonds?a...,289,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 47.8315724, 'longitude': -122.336...","[pickup, delivery]",$$,"{'address1': '7533 Olympic View Dr', 'address2...",+14253617640,(425) 361-7640,10346.304888
2,njvyS8D09NfOhIDbEjNhEg,larrys-smokehouse-snohomish,Larry's Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/7R7k4X...,False,https://www.yelp.com/biz/larrys-smokehouse-sno...,176,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 47.839407, 'longitude': -122.124891}","[pickup, delivery]",$,"{'address1': '17416 State Route 9', 'address2'...",+14254837001,(425) 483-7001,5611.595646
3,f4rnFr_bFWOB44crXfFMbQ,j-and-l-bbq-snohomish,J & L BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/2z6_9B...,False,https://www.yelp.com/biz/j-and-l-bbq-snohomish...,298,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 47.91278, 'longitude': -122.09882}","[pickup, delivery]",$$,"{'address1': '130 Ave D', 'address2': None, 'a...",+13605687222,(360) 568-7222,10622.755887
4,GAq0SZKYEeWN71hC0FHYrQ,j-and-l-bbq-everett,J & L BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/OTdSHc...,False,https://www.yelp.com/biz/j-and-l-bbq-everett?a...,137,"[{'alias': 'tradamerican', 'title': 'American'...",4.5,"{'latitude': 47.97862, 'longitude': -122.20819}",[delivery],$$,"{'address1': '2915 Colby Ave', 'address2': '',...",+14252593222,(425) 259-3222,14830.981444
5,D_ClecX3hizRwnQyamtf6g,carolina-smoke-bbq-bothell,Carolina Smoke BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/NqM0cv...,False,https://www.yelp.com/biz/carolina-smoke-bbq-bo...,484,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"{'latitude': 47.7784982305512, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '24118 Bothell Everett Hwy', 'add...",+14259498672,(425) 949-8672,7573.454942
6,6jr4aZ1eMLadC0FntrXKgw,dickeys-barbecue-pit-everett-3,Dickey's Barbecue Pit,https://s3-media3.fl.yelpcdn.com/bphoto/mSZAuS...,False,https://www.yelp.com/biz/dickeys-barbecue-pit-...,209,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 47.88238744275234, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '617 SW 128th St', 'address2': 'S...",+14252650041,(425) 265-0041,5180.721616
7,RTgQWV2_XxraHo5b-88kaw,guam-grub-everett,Guam Grub,https://s3-media2.fl.yelpcdn.com/bphoto/Lvbqkj...,False,https://www.yelp.com/biz/guam-grub-everett?adj...,27,"[{'alias': 'guamanian', 'title': 'Guamanian'},...",4.5,"{'latitude': 47.908545696227335, 'longitude': ...",[],NaN,"{'address1': '1402 SE Everett Mall Way', 'addr...",+14253089997,(425) 308-9997,7146.156786
8,RmLxbaA32FB0D6l3Rm-TnA,best-of-both-worlds-snohomish,Best of Both Worlds,https://s3-media4.fl.yelpcdn.com/bphoto/BuzAXU...,False,https://www.yelp.com/biz/best-of-both-worlds-s...,16,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.5,"{'latitude': 47.934462, 'longitude': -122.105021}","[pickup, delivery]",NaN,"{'address1': '1830 Bickford Ave', 'address2': ...",+14252725386,(425) 272-5386,12160.356077
9,aGyck-ohu3pnwtWHgCEHKQ,wake-n-bacon-bbq-shoreline-2,Wake N Bacon BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/sHJ0At...,False,https://www.yelp.com/biz/wake-n-bacon-bbq-shor...,122,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 47.7693646155242, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '1437 NW 

- Where is the actual data we want to save?

In [44]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [45]:
(results['total'])/results_per_page

36.65

In [46]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

37

In [47]:
for i in tqdm_notebook(range(1, n_pages+1)):
    ## The block of code we want to TRY to run
    try:

        time.sleep(.2)
        

        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term, offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)


  0%|          | 0/37 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [48]:
df = pd.read_json(JSON_FILE)

In [49]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,njvyS8D09NfOhIDbEjNhEg,larrys-smokehouse-snohomish,Larry's Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/7R7k4X...,False,https://www.yelp.com/biz/larrys-smokehouse-sno...,176,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",3.5,"{'latitude': 47.839407, 'longitude': -122.124891}","[pickup, delivery]",$,"{'address1': '17416 State Route 9', 'address2'...",+14254837001,(425) 483-7001,5611.595646
1,ZIRJB-lk5cD4L8GUOca3zw,fat-pig-bbq-edmonds,Fat Pig BBQ,https://s3-media2.fl.yelpcdn.com/bphoto/Xdbg4J...,False,https://www.yelp.com/biz/fat-pig-bbq-edmonds?a...,289,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 47.8315724, 'longitude': -122.336...","[pickup, delivery]",$$,"{'address1': '7533 Olympic View Dr', 'address2...",+14253617640,(425) 361-7640,10346.304888
2,f4rnFr_bFWOB44crXfFMbQ,j-and-l-bbq-snohomish,J & L BBQ,https://s3-media1.fl.yelpcdn.com/bphoto/2z6_9B...,False,https://www.yelp.com/biz/j-and-l-bbq-snohomish...,298,"[{'alias': 'bbq', 'title': 'Barbeque'}]",4.5,"{'latitude': 47.91278, 'longitude': -122.09882}","[pickup, delivery]",$$,"{'address1': '130 Ave D', 'address2': None, 'a...",+13605687222,(360) 568-7222,10622.755887
3,GAq0SZKYEeWN71hC0FHYrQ,j-and-l-bbq-everett,J & L BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/OTdSHc...,False,https://www.yelp.com/biz/j-and-l-bbq-everett?a...,137,"[{'alias': 'tradamerican', 'title': 'American'...",4.5,"{'latitude': 47.97862, 'longitude': -122.20819}",[delivery],$$,"{'address1': '2915 Colby Ave', 'address2': '',...",+14252593222,(425) 259-3222,14830.981444
4,D_ClecX3hizRwnQyamtf6g,carolina-smoke-bbq-bothell,Carolina Smoke BBQ,https://s3-media3.fl.yelpcdn.com/bphoto/NqM0cv...,False,https://www.yelp.com/biz/carolina-smoke-bbq-bo...,484,"[{'alias': 'bbq', 'title': 'Barbeque'}]",3.5,"{'latitude': 47.7784982305512, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '24118 Bothell Everett Hwy', 'add...",+14259498672,(425) 949-8672,7573.454942


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             732 non-null    object 
 1   alias          732 non-null    object 
 2   name           732 non-null    object 
 3   image_url      732 non-null    object 
 4   is_closed      732 non-null    bool   
 5   url            732 non-null    object 
 6   review_count   732 non-null    int64  
 7   categories     732 non-null    object 
 8   rating         732 non-null    float64
 9   coordinates    732 non-null    object 
 10  transactions   732 non-null    object 
 11  price          636 non-null    object 
 12  location       732 non-null    object 
 13  phone          732 non-null    object 
 14  display_phone  732 non-null    object 
 15  distance       732 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 86.6+ KB


In [51]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json', '.csv.gz')
csv_file

'Data/Seattle-barbecue.csv.gz'

In [52]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [53]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 720,602 Bytes
CSV.GZ FILE: 103,881 Bytes
the csv.gz is 6.936802687690723 times smaller!


## Next Class: Processing the Results and Mapping 